#### Airbnb - Part 3 of 3 (Preliminary Modells - XGBoost)

Running a prelimnary model on the data cleaned and anlysed so far in part 1 and part 2 (ensemble models). Advanced Modelling (Part 1 & 2) to be uploaded in the coming week. I made use of some great suggestions made in kaggle kernels, the references for which are listed below.

In [5]:
# Import relevant libraraies
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os

In [32]:
# Load the data from Part 2
train = pd.read_csv("train_users_4.csv")
test = pd.read_csv("test_users.csv")
train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19,NaN,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,2009-05-23,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,2009-06-09,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,2009-10-31,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,2009-12-08,2010-02-18,NaN,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [33]:
country = pd.read_csv('countries.csv')
country.head()

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06


In [34]:
test_ids = test['id']
Nrows_train = train.shape[0]  

# Store country names
labels = train['country_destination'].values
train1 = train.drop(['country_destination'], axis=1)

# Combining the test and train data. If this is not done, the number of dummy variable columns do not match in test and train data.
# Some items present in train data and are not present in test data. For example, browser type. 
data_all = pd.concat((train1,test), axis = 0, ignore_index = True)

# Dropping ids which are saved separately and date of first booking which is completely absent in the test data
data_all = data_all.drop(['id','date_first_booking'], axis=1)

In [35]:
data_all.head()

,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,2010-06-28,2009-03-19,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome
1,2011-05-25,2009-05-23,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome
2,2010-09-28,2009-06-09,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE
3,2011-12-05,2009-10-31,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox
4,2010-09-14,2009-12-08,NaN,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome


In [36]:
print(data_all.isnull().sum())

date_account_created            0
timestamp_first_active          0
gender                      95688
age                        119462
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6085
signup_app                      0
first_device_type               0
first_browser               27266
dtype: int64


In [37]:
data_all.gender.replace('-unknown-', np.nan, inplace=True)
data_all.first_browser.replace('-unknown-', np.nan, inplace=True)

In [38]:
data_all.loc[data_all.age > 85, 'age'] = np.nan
data_all.loc[data_all.age < 18, 'age'] = np.nan

In [39]:
print(data_all.isnull().sum())

date_account_created            0
timestamp_first_active          0
gender                     129480
age                        120095
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6085
signup_app                      0
first_device_type               0
first_browser               44394
dtype: int64


In [40]:
data_all.date_account_created = pd.to_datetime(data_all.date_account_created)
data_all.timestamp_first_active = pd.to_datetime(data_all.timestamp_first_active)

In [41]:
# Splitting date time data for date account created
data_all['dac_year'] = data_all.date_account_created.dt.year
data_all['dac_month'] = data_all.date_account_created.dt.month
data_all['dac_day'] = data_all.date_account_created.dt.day

# Splitting date time data for time first active
data_all['tfa_year'] = data_all.timestamp_first_active.dt.year
data_all['tfa_month'] = data_all.timestamp_first_active.dt.month
data_all['tfa_day'] = data_all.timestamp_first_active.dt.day

data_all.drop('date_account_created',1, inplace=True)
data_all.drop('timestamp_first_active',1, inplace=True)

In [42]:
data_all.head()

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day
0,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,6,28,2009,3,19
1,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2011,5,25,2009,5,23
2,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2010,9,28,2009,6,9
3,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2011,12,5,2009,10,31
4,NaN,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,9,14,2009,12,8


In [43]:
data_all.describe()

,age,signup_flow,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day
count,155452.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000
mean,35.942362,4.291965,2013.243828,6.457505,15.958969,2003.327705,4.890563,12.518246
std,11.346129,8.794313,0.921206,2.985339,8.757489,17.994818,3.538048,9.887532
min,18.000000,0.000000,2010.000000,1.000000,1.000000,1970.000000,1.000000,1.000000
25%,28.000000,0.000000,2013.000000,4.000000,8.000000,2011.000000,1.000000,2.000000
50%,33.000000,0.000000,2014.000000,7.000000,16.000000,2013.000000,4.000000,12.000000
75%,41.000000,1.000000,2014.000000,9.000000,23.000000,2014.000000,8.000000,21.000000
max,85.000000,25.000000,2014.000000,12.000000,31.000000,2014.000000,12.000000,31.000000


In [44]:
print(data_all.isnull().sum())

gender                     129480
age                        120095
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6085
signup_app                      0
first_device_type               0
first_browser               44394
dac_year                        0
dac_month                       0
dac_day                         0
tfa_year                        0
tfa_month                       0
tfa_day                         0
dtype: int64


In [45]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a MinMax scaler, then apply it to the numerical features
scaler = MinMaxScaler()
numerical = ['age','dac_year','dac_month','dac_day','tfa_year','tfa_month','tfa_day']
data_all[numerical] = scaler.fit_transform(data_all[numerical])

In [46]:
# Create categorical columns
features = ['gender','signup_method','signup_flow','language','affiliate_channel','affiliate_provider',\
            'first_affiliate_tracked','signup_app','first_device_type','first_browser']

# get dummies
data_all = pd.get_dummies(data_all,columns=features)

In [47]:
data_all.describe()

,age,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day,gender_FEMALE,gender_MALE,gender_OTHER,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
count,155452.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,...,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000,275547.000000
mean,0.267796,0.810957,0.496137,0.498632,0.757448,0.353688,0.383942,0.281346,0.247540,0.001212,...,0.000624,0.000098,0.000007,0.000156,0.000004,0.000029,0.000007,0.000004,0.000051,0.000025
std,0.169345,0.230302,0.271394,0.291916,0.408973,0.321641,0.329584,0.449657,0.431584,0.034795,...,0.024976,0.009898,0.002694,0.012491,0.001905,0.005388,0.002694,0.001905,0.007128,0.005040
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.149254,0.750000,0.272727,0.233333,0.931818,0.000000,0.033333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.223881,1.000000,0.545455,0.500000,0.977273,0.272727,0.366667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.343284,1.000000,0.727273,0.733333,1.000000,0.636364,0.666667,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
# Splitting train and test for the classifier
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder

V = data_all.values
X_train = V[:Nrows_train]
X_test = V[Nrows_train:]

#Create labels
labler = LabelEncoder()
y = labler.fit_transform(labels)

# Implementation of the classifier (decision tree)
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=22,
                    objective='multi:softprob', subsample=0.6, colsample_bytree=0.6, seed=0)               
xgb.fit(X_train, y)
y_pred = xgb.predict_proba(X_test) 

In [52]:
y_pred

array([[0.00416745, 0.00824285, 0.00592712, ..., 0.00266366, 0.30068907,
        0.03478919],
       [0.00132019, 0.00366531, 0.00180506, ..., 0.00122972, 0.12381469,
        0.01880966],
       [0.00246855, 0.0094469 , 0.00416053, ..., 0.00234134, 0.1893225 ,
        0.03137904],
       ...,
       [0.00375343, 0.01080159, 0.00697793, ..., 0.00238036, 0.18380354,
        0.02069318],
       [0.00303393, 0.00770491, 0.00504528, ..., 0.00244463, 0.1770713 ,
        0.02006082],
       [0.00421506, 0.01170511, 0.00642884, ..., 0.00276177, 0.5323202 ,
        0.05009951]], dtype=float32)

In [53]:
#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(test_ids)):
    idx = test_ids[i]
    ids += [idx] * 5
    cts += labler.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

In [54]:
#Generate submission
submission = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
submission.to_csv('submission.csv',index=False)

#### References

1. https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
2. https://www.kaggle.com/kanncaa1/data-sciencetutorial-for-beginners
3. https://www.kaggle.com/justk1/airbnb
4. https://www.kaggle.com/davidgasquez/user-data-exploration
5. https://www.kaggle.com/kevinwu06/airbnb-exploratory-analysis
6. https://www.kaggle.com/rounakbanik/airbnb-new-user-bookings

##### - Aditya